In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
# %matplotlib notebook
%matplotlib inline
sns.set()

from util import DataFrameBase, xlim_expand
from tbtools.dev import IProgressBar, ProgressBar, subreload
from tbtools.iter import replace_repetitions, tbfilter

In [ ]:
figspath = os.environ['HOME'] + '/Dropbox/DTU/4th Semester/figs/'

In [ ]:
db_parent = os.environ['HOME'] + '/Speciale/data/exported/'
dbs = ('NN_1_10', 'NN_26', 'NN_CLC')

# https://docs.python.org/3/library/codecs.html#standard-encodings
global_settings = {
    'delimiter':';', 
    'encoding':'cp1252',
    'infer_datetime_format':True,
}

In [ ]:
def parse_dates(*args):
    return {'parse_dates': list(args)}

datotid = parse_dates('DatoTid')
timestamp = parse_dates('Timestamp')

In [ ]:
# All three DBs have the same tables
s

clc = DataFrameBase(db_parent, 'clc', 
                    global_settings=global_settings, 
                    table_settings=table_settings)

b1 = DataFrameBase(db_parent, '1', 
                   global_settings=global_settings, 
                   table_settings=table_settings)

b2 = DataFrameBase(db_parent, '2', 
                   global_settings=global_settings, 
                   table_settings=table_settings)

## Check normalization method for whether it is ok

In [ ]:
df.to_sql?

In [ ]:
def normalize_df(df, intended_index):
    """df is a dataframe
    intended_index is a column name; this column will
        be used as index in the normalized df. If not
        defined, no new index is set.
    
    Discards rows that have all the same information, 
        including timestamps.
    Discards rows and column with only NaNs.
    """
    
    df = df.dropna(0, 'all').dropna(1, 'all')
    df.drop_duplicates(inplace=True)
    if intended_index:
        df.set_index(intended_index, verify_integrity=True, inplace=True)
    df.sort_index(inplace=True)
    
    return df

In [ ]:
df = pd.DataFrame([["xx",1,2,3], ["f", 1,np.nan,4], ["j", 1,np.nan,4]], 
                  columns=list('abcd'), 
                  index=[1,2,2])
normalize_df(df, 'a')

In [ ]:
ah = clc['AlmHist'].set_index('DatoTid')

In [ ]:
%%time
norm = normalize_df(ah)

In [ ]:
ah.head()

In [ ]:
len(ah.index.unique())

In [ ]:
len(norm.index.unique())

In [ ]:
len(norm.ix[ah.index])

In [ ]:
len(ah)

In [ ]:
len(norm)

In [ ]:
ah.AlmState == norm.ix[ah.index].AlmState

In [ ]:
norm.head()

---

# Compare date ranges

In [ ]:
dbs = (clc, b1, b2)

Load all known DateTime columns into a list per database

In [ ]:
timecols = dict([(db.name, []) for db in dbs])

for k in IProgressBar(table_settings):
    datecols = table_settings[k]['parse_dates']
    for db in dbs:
        for v in datecols:
            timecols[db.name].append(db[k][v])

For each db, concatenate its time columns to one long column,
and resample it to be the number of timecol entries per day.

Dub this the `activity`.

In [ ]:
activity = {}

for k in IProgressBar(timecols):
    ser = pd.concat(timecols[k], ignore_index=True)
    df = pd.concat([ser, pd.Series(np.ones(ser.shape))], axis=1).set_index(0)
    df.columns = [k]
    df.fillna(0, inplace=True)
    df = df.resample('D', 'sum')
    activity[k] = df

Put the number of times each day appears in each db together in a DataFrame

Dub it `day_seen`

In [ ]:
day_seen = pd.concat(activity.values(), axis=1)

In [ ]:
def activation(df):
    """df is a dataframe
    each column holds counts
    This is turned into an indicator: > 0 or not
    And the values in each
    """
    df[df.gt(0)] = 1
    for i, col in enumerate(df):
        df[col] *= i+1
    df[df.le(0)] = np.float('nan')
    return df    

Make indicator variables to plot the dates with activity for each database

In [ ]:
indicator = activation(day_seen)

In [ ]:
sns.plt.savefig?

In [ ]:
plot = indicator.plot(style='.', ylim=(0,4), alpha=0.5, figsize=(12,4))
# plot.set_title('Dates covered by timestamps in the three initial databases')
xlim_expand(plot, 15)
plot.set_xlabel('')
plot.tick_params(
    axis='y',        # changes apply to the x-axis
    which='both',    # both major and minor ticks are affected
    left='off',      # ticks along the bottom edge are off
    right='off',     # ticks along the top edge are off
    labelleft='off') # labels along the left edge are off
sns.plt.tight_layout()
sns.plt.savefig(figspath + '3db_covered.png')

In [ ]:
ts_differ = mdf[(mdf.iloc[:,0] != mdf.iloc[:,1]) | 
                (mdf.iloc[:,2] != mdf.iloc[:,1]) |
                (mdf.iloc[:,0] != mdf.iloc[:,2])].stack(-1)

plot = ts_differ.unstack().plot(style='o', alpha=0.5,  figsize=(12,4))
# plot.set_title('Number of timestamps per database for days where they differ')
plot.set_ylabel('Number of timestamps on day')
plot.set_xlabel('')
xlim_expand(plot, 10)
sns.plt.tight_layout()
sns.plt.savefig(figspath + '3db_difference.png')

# Consider the differences in the week in the beginning of July

In [ ]:
ahc = clc.AlmHist.set_index('DatoTid')
ah1 = b1.AlmHist.set_index('DatoTid')
ah2 = b2.AlmHist.set_index('DatoTid')

In [ ]:
df = clc.AlmHist

In [ ]:
def aggregate_activity(resample_how='D', default_value=0, **dfs):
    """Takes named dataframes with DateTime index, and
    returns a single dataframe with an indicator variable
    for each input df, showing whether that df had an entry
    in the time slot.
    """
    mindt, maxdt = None, None
    for n in dfs:
        s = dfs[n].index
        if mindt is None:
            mindt, maxdt = s.min(), s.max()
            continue
        mindt = min(mindt, s.min())
        maxdt = max(maxdt, s.max())
    
    index = pd.date_range(mindt, maxdt, freq=resample_how, normalize=True)
    
    df = pd.DataFrame({col:default_value for col in dfs}, index=index)
    
    for n in dfs:
#         s = (dfs[n].resample(resample_how, 'count') > 0).max(axis=1)
        s = (dfs[n].resample(resample_how, 'count')).max(axis=1)
        df.loc[s.index, n] = s
        
    agree = np.ones(len(df))
    for i, (label, v) in enumerate(df.iterrows()):
        if not all(v == v[0]):
            agree[i] = 0
            continue
        if all(v == 0):
            continue
        subsets = [df[str(label.date())] for df in dfs.values()]
        if not functionally_equal(subsets):
            agree[i] = 0
        
    df['agree'] = agree * df.max().max()
#     compare_content(list(dfs.values()), index) * df.max().max()
        
    return df

def compare_activity(resample_how='D', title='no title', **dfs):
    df = aggregate_activity(**dfs, resample_how=resample_how)
    
    labels = [d.strftime('%b %d') for i,d in enumerate(df.index)]
    labels = list(tbfilter(labels, [1,0], repl=['']))
    
    figwidth = len(df)/1.5#3.5
    sns.plt.figure(figsize=(figwidth,5))
    cm = sns.heatmap(df.T, 
                     xticklabels=labels,
                     linewidths=0.5,
                     cbar=False,
                     square=True,
                     annot=True, fmt='.0f')
    cm.set_title(title)
    cm.set_xlabel('Date in 2012')

dbs = (('clc', clc), ('b1',b1), ('b2',b2))

def functionally_equal(dfs):
    dfs = iter(normalize_df(df) for df in dfs)
    canon = next(dfs)
    for d in dfs:
        try:
            pd.util.testing.assert_frame_equal(canon, d)
        except AssertionError:
            return False
    return True
    

def compare_content(dfs, major_index):
    """Takes a list of dataframes
    Outputs a list indicating for each item in major_index
    whether the content in all of them is exactly the same, 
    after sorting index, removing nans, and removing duplicates.
    """
    dfs = list(map(normalize_df, dfs))
    
    labels = [str(i.date()) for i in major_index]
    
    output = np.ones(len(major_index))
    
    for i, label in enumerate(labels):
        continue
        has_label = [(label in d.index) for d in dfs]
        
        if not all(has_label):
            if any(has_label):
                # Some dfs have data under this label, while
                # other do not. They do not agree.
                output[i] = 0
            continue
        
        parts = iter(df[label] for df in dfs)
        canon = next(parts)
        
        for p in parts:
            try:
                pass
#                 pd.util.testing.assert_frame_equal(canon, p)
            except AssertionError:
                output[i] = 0
                break
        
#         if not all((len(canon) == len(p)) for p in parts[1:]):
#             output[i] = 0#or \
#             continue
#         if not all((canon.index == p.index).all() for p in parts[1:]):
#             output[i] = 0#or \
#             continue
#         if not all((canon == p).all().all() for p in parts[1:]):
#             output[i] = 0

    return output


def normalize_df(df):
    indexname = df.index.name or 'index'
    
    # make sure to take indices into account when dropping dupes
    return df.fillna(0).\
              reset_index().\
              drop_duplicates().\
              set_index(indexname).\
              sort_index()
        
def compare_table(table_name):
    pds = table_settings[table_name]['parse_dates']
    for pd in pds:
        title = '{}: {}'.format(table_name, pd)
        tabs = {n:db[table_name].set_index(pd) for n,db in dbs}
        lengths = [len(tab) for tab in tabs.values()]

        if all([L==0 for L in lengths]):
            print('Skipping {}: No data.'.format(table_name))
            return
        
        compare_activity(title=title, **tabs)
    
def compare_all_tables():
    for t in IProgressBar(table_settings):
#     for t in table_settings:
        compare_table(t)

In [ ]:
compare_table('AlmHist')

In [ ]:
clc.tables

In [ ]:
clc.efterkontrol

In [ ]:
df = b1.AlmHist

In [ ]:
b2.RaavareDB.plot(x='Timestamp', y='Vaegt_Bemaerkning', style='.')

In [ ]:
# %debug
compare_all_tables()

In [ ]:
compare_all_tables() # with soft counts

### Look at Sep 01 (or 02, 03, 04, 05) for AlmHist in all three databases. How do they differ?

In summary, they are exactly equal, when you remember to fillna and sort_index.

In [ ]:
day = '2012-09-01'
tables = [('clc', ahc), ('b1', ah1), ('b2', ah2)]

def prep(df):
    return df.sort_index().fillna(-1)

In [ ]:
def evaluate_equality(day):
    dfs = [(name, prep(d[day])) for name,d in tables]
    
    eq = []
    for (n1, df1), (n2, df2) in combinations(dfs, 2):
        try:
            e = all(df1 == df2)
        except:
            e = False
        eq.append(e)
        if not e:
            print('{} and {} not equal on {}'.format(n1, n2, day))
    
    print('All equal on day {}?'.format(day), all(eq))

In [ ]:
for day in pd.date_range('2012-09-01', '2012-09-08'):
    evaluate_equality(str(day.date()))

In [ ]:
print(len(hc), len(h1), len(h2))

- Where the datetime indices match, i.e. where the timestamps are the same, is the data also the same?
- yes.
    + For B1 and B2, the data is exactly the same for the whole day.
    + When they are sorted by index, so that the rows are ordered by their DatoTid, everything is the same.
    + False earlier conclusion: *For CLC and the others, the overlap is 1038 perfect matches of 1302, where the 264 remaining have odd timestamps.*

In [ ]:
day = '2012-09-02'

all(hc.sort_index().fillna(0) == h1.sort_index().fillna(0))

In [ ]:
from itertools import combinations

dfs = [('clc', hc.sort_index().fillna(-1)), ('b1', h1.sort_index().fillna(-1)), ('b2', h2.sort_index())]
# Where the indices match, are they equal?
for (n1, df1), (n2, df2) in combinations(dfs, 2):
    eq = df1.index == df2.index
    print('Comparing {} and {} AlmHist on {}:'.format(n1, n2, day), 
          all(df1[eq].fillna(0) == df2[eq].fillna(0)))
    print('\tOverlap is {}/{}'.format(sum(eq), len(df1)))
    
# eq = h1.index == hc.index
# print('Comparing b1 and clc AlmHist on {}:'.format(day), all(h1[eq].fillna(0) == hc[eq].fillna(0)))
# print('Overlap is {}/{}'.format(sum(eq), len(h1)))
# eq = h2.index == hc.index
# print('Comparing b2 and clc AlmHist on {}:'.format(day), all(h2[eq].fillna(0) == hc[eq].fillna(0)))
# print('Overlap is {}/{}'.format(sum(eq), len(h2)))
# print('Comparing b2 and clc AlmHist on {}:'.format(day), all(h2[eq].fillna(0) == hc[eq].fillna(0)))
# print('Overlap is {}/{}'.format(sum(eq), len(h2)))

## With correct fillna and sort_index, how often do the three databases concur?

In [ ]:
hc = clc.AlmHist.set_index('DatoTid').sort_index().fillna(-1)
h1 = b1.AlmHist.set_index('DatoTid').sort_index().fillna(-1)
h2 = b2.AlmHist.set_index('DatoTid').sort_index().fillna(-1)

- According to index.isin,
    + h1 is a superset of h2
    + hc is a superset of neither, or at least doesn't completely contain either.

In [ ]:
print('hc number of dupes {}'.format(len(hc) - len(hc.reset_index().drop_duplicates())))
print('h1 number of dupes {}'.format(len(h1) - len(h1.reset_index().drop_duplicates())))
print('h2 number of dupes {}'.format(len(h2) - len(h2.reset_index().drop_duplicates())))

In [ ]:
print('hc number of nan {}'.format(len(hc) - len(hc.dropna())))
print('h1 number of nan {}'.format(len(h1) - len(h1.dropna())))
print('h2 number of nan {}'.format(len(h2) - len(h2.dropna())))

# How to merge two dataframes with similar data

In [ ]:
df1 = pd.DataFrame(np.array([[1,2,4],[4,5,6], [1,1,1]]), columns=['a','b','c'], index=[1,2,15])
df2 = pd.DataFrame(np.array([[1,2,3],[1,1,1], [14,15,16]]), columns=['a','b','c'], index=[1,15, 3])

res = df1.align(df2) # Get df1 and df2, with aligned indices and lots of NaNs
res = pd.concat(res, join='outer') # Concatenate them
# Clean up
res = res.dropna()
res = res.sort_index()

res = res.drop_duplicates()
res

----

## What do we get when we align-concat-dropna-dropdupes on the three AlmHist tables?

In [ ]:
print('Raw len {} {}'.format(len(hc), len(h1)))
res = hc.align(h1) # Get df1 and df2, with aligned indices and lots of NaNs
print('aligned len {}'.format(len(res[0])))
res = pd.concat(res, join='outer') # Concatenate them
print('concat len {}'.format(len(res)))
# Clean up
res = res.dropna()
print('dropped na len {}'.format(len(res)))
res = res.sort_index()
indexname = res.index.name or 'index'
res = res.reset_index().drop_duplicates().set_index(indexname)
print('dropped duplicates len {}'.format(len(res)))
# res

It appears that hc and h1 have an overlap of 

sum of original lenghts - final length $= (3087964+829529) - 3834292 = 83201$

In [ ]:
print('merging h2 into the merged hc h1')
print('Raw len {} {}'.format(len(res), len(h2)))
res2 = res.align(h2)
print('aligned len {}'.format(len(res2[0])))
res2 = pd.concat(res2, join='outer') # Concatenate them
print('concat len {}'.format(len(res2)))
# Clean up
res2 = res2.dropna()
print('dropped na len {}'.format(len(res2)))
res2 = res2.sort_index()
indexname = res2.index.name or 'index'
res2 = res2.reset_index().drop_duplicates().set_index(indexname)
print('dropped duplicates len {}'.format(len(res2)))
# res

h2 had nothing to add.

In [ ]:
(res.resample('D', 'count') > 0).plot(style='.', legend=False)